# Imports

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd 
import os

In [3]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from datetime import datetime as dt
import pytz

In [4]:
PATH = 'data/nyc-taxi/'
CSV_PATH = 'submissions/v17_ensemble_16trees_54M/'

# Helper functions

In [5]:
def split_df(df, y_fld, subset=None, random_state=42):
    
    if subset: df = df.sample(n=subset, random_state=random_state)

    y = df[y_fld].values
    df.drop(y_fld, axis=1, inplace=True)
    
    return df, y

# Load data

In [6]:
seed = 101 # random seed for replication

In [7]:
train_df = pd.read_feather('tmp/taxi-train-v10-Airport-Flag')
test_df = pd.read_feather('tmp/taxi-test-v10-Airport-Flag')
test_df_raw = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

In [8]:
train_df.shape, test_df.shape

((54075311, 25), (9914, 24))

In [9]:
X_train, y_train = split_df(train_df, 'fare_amount')

In [10]:
X_train.shape, y_train.shape, test_df.shape

((54075311, 24), (54075311,), (9914, 24))

# Batch fit

In [11]:
outdir = CSV_PATH
os.makedirs(outdir, exist_ok=True)
preds = []
n_forest=1
tz = pytz.timezone('Asia/Singapore')

In [12]:
%%time
for i in range (n_forest):
    
    new_seed = i
    
    print(f"Random Forest [{i+1}/{n_forest}] of 16 trees with random seed {new_seed}")
    m = RandomForestRegressor(n_estimators=16, min_samples_leaf=3, max_features=0.5, n_jobs=-1, random_state=new_seed)
    
    print("Fitting on training data...")
    m.fit(X_train, y_train)

    print("Predicting on test data...")
    test_y_predictions = m.predict(test_df)
    preds.append(test_y_predictions)      
    
    print("Creating submission file...")
    
    cur_dt = dt.now(tz).strftime("%Y%m%d_%H%M%S")
    desc = 'v17_RF_' # description of submission for reference
    ensemb_desc = '_01x16_54M'
    subm_fn =  desc + cur_dt + ensemb_desc + '.csv'
    subm_path = f'{outdir}/{subm_fn}'

    # Write the predictions to a CSV file which we can submit to the competition.
    submission = pd.DataFrame(
        {'key': test_df_raw.key, 'fare_amount': test_y_predictions},
        columns = ['key', 'fare_amount'])
    submission.to_csv(subm_path, index = False)
    
    print(f"Submission: {subm_fn}")
    print()

Random Forest [1/1] of 16 trees with random seed 0
Fitting on training data...
Predicting on test data...
Creating submission file...
Submission: v17_RF_20180813_171945_01x16_54M.csv

CPU times: user 14h 15min 34s, sys: 1min 33s, total: 14h 17min 7s
Wall time: 57min 19s


# Results

In [ ]:
Seed 100, 101, 102